In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import shutil

In [2]:
data = pd.read_csv("names.csv")
data = data['Cars'].str.split(expand=True)
data = data.iloc[:, :-2]
data.columns = ["Brand" , "Name" , "Type","Year"]
data.head()

,Brand,Name,Type,Year
0,AM-General,Hummer,SUV,2000
1,Acura,RL,Sedan,2012
2,Acura,TL,Sedan,2012
3,Acura,TL,Type-S,2008
4,Acura,TSX,Sedan,2012


In [3]:
import os

source_folder = "Train"  # Specify your source folder

for folder_name in os.listdir(source_folder):
    # Check if it's a directory
    if os.path.isdir(os.path.join(source_folder, folder_name)):
        # Extract the first word from the folder name
        first_word = folder_name.split()[0]
        
        # Create the destination folder if it doesn't exist
        dest_folder = os.path.join("TrainData", first_word)
        os.makedirs(dest_folder, exist_ok=True)
        
        # Move the folder to the destination folder
        shutil.move(os.path.join(source_folder, folder_name), dest_folder)
        print(f"Moved {folder_name} to {first_word}")

print("Move operation completed.")


Moved Acura Integra Type R 2001 to Acura
Moved Acura RL Sedan 2012 to Acura
Moved Acura TL Sedan 2012 to Acura
Moved Acura TL Type-S 2008 to Acura
Moved Acura TSX Sedan 2012 to Acura
Moved Acura ZDX Hatchback 2012 to Acura
Moved AM-General Hummer SUV 2000 to AM-General
Moved Aston Martin V8 Vantage Convertible 2012 to Aston
Moved Aston Martin V8 Vantage Coupe 2012 to Aston
Moved Aston Martin Virage Convertible 2012 to Aston
Moved Aston Martin Virage Coupe 2012 to Aston
Moved Audi 100 Sedan 1994 to Audi
Moved Audi 100 Wagon 1994 to Audi
Moved Audi A5 Coupe 2012 to Audi
Moved Audi R8 Coupe 2012 to Audi
Moved Audi RS 4 Convertible 2008 to Audi
Moved Audi S4 Sedan 2007 to Audi
Moved Audi S4 Sedan 2012 to Audi
Moved Audi S5 Convertible 2012 to Audi
Moved Audi S5 Coupe 2012 to Audi
Moved Audi S6 Sedan 2011 to Audi
Moved Audi TT Hatchback 2011 to Audi
Moved Audi TT RS Coupe 2012 to Audi
Moved Audi TTS Coupe 2012 to Audi
Moved Audi V8 Sedan 1994 to Audi
Moved Bentley Arnage Sedan 2009 to Bentl

In [4]:
label_map = {}

In [5]:
target_size = (224, 224)
# Initialize lists to store images and labels
images = []
labels = []
i=0
for root, dirs, files in os.walk("Train"):
    for folder in dirs:
        folder_path = os.path.join(root, folder)
        # Get label based on folder name
        label_map[folder] = i 
        label = i
        i +=1
        # Use folder name as label
        # Iterate over each file inside the folder
        for filename in os.listdir(folder_path):
            if filename.endswith('.jpg'):
                # Load and preprocess the image
                img = tf.keras.preprocessing.image.load_img(
                    os.path.join(folder_path, filename), target_size=target_size
                )
                img_array = tf.keras.preprocessing.image.img_to_array(img)
                img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
                
                # Add the preprocessed image to the list of images and corresponding label to labels
                images.append(img_array)
                labels.append(label)

# Convert lists to numpy arrays
X_train = np.array(images)
Y_train = np.array(labels)

In [18]:
target_size = (224, 224)
images = []
labels = []
label_map = {}
i = 0

for root, dirs, files in os.walk("train"):
    for folder in dirs:
        # Extract the first word from the folder name
        first_word = folder.split()[0]
        # Assign a label to the first word if it hasn't been assigned before
        if first_word not in label_map:
            label_map[first_word] = i
            i += 1

        # Get the label for the current folder
        label = label_map[first_word]

        folder_path = os.path.join(root, folder)
        # Iterate over each file inside the folder
        for filename in os.listdir(folder_path):
            if filename.endswith('.jpg'):
                # Load and preprocess the image
                img = tf.keras.preprocessing.image.load_img(
                    os.path.join(folder_path, filename), target_size=target_size
                )
                img_array = tf.keras.preprocessing.image.img_to_array(img)
                img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
                
                # Add the preprocessed image to the list of images and corresponding label to labels
                images.append(img_array)
                labels.append(label)

# Convert lists to numpy arrays
X_train = np.array(images)
Y_train = np.array(labels)


In [19]:
X_train.shape

(8144, 224, 224, 3)

In [23]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),weights="imagenet",include_top=False)

In [24]:
x = tf.keras.layers.GlobalAvgPool2D()(base_model.output)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128,activation="relu")(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(64,activation="relu")(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(196,activation="softmax")(x)

model = tf.keras.Model(inputs = base_model.input , outputs = x)


In [25]:
#freezing the original model's layers
for layer in base_model.layers:
    layer.trainable = False

In [26]:
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])

In [27]:
model.fit(X_train,Y_train,epochs = 10)

Epoch 1/10
255/255 ━━━━━━━━━━━━━━━━━━━━ 105s 369ms/step - accuracy: 0.1054 - loss: 3.9793
Epoch 2/10
255/255 ━━━━━━━━━━━━━━━━━━━━ 97s 382ms/step - accuracy: 0.2158 - loss: 2.7936
Epoch 3/10
255/255 ━━━━━━━━━━━━━━━━━━━━ 100s 394ms/step - accuracy: 0.2942 - loss: 2.4657
Epoch 4/10
255/255 ━━━━━━━━━━━━━━━━━━━━ 100s 392ms/step - accuracy: 0.3337 - loss: 2.2552
Epoch 5/10
255/255 ━━━━━━━━━━━━━━━━━━━━ 108s 423ms/step - accuracy: 0.3853 - loss: 2.0717
Epoch 6/10
255/255 ━━━━━━━━━━━━━━━━━━━━ 104s 409ms/step - accuracy: 0.4160 - loss: 1.9339
Epoch 7/10
104/255 ━━━━━━━━━━━━━━━━━━━━ 57s 382ms/step - accuracy: 0.4772 - loss: 1.7905

KeyboardInterrupt: 

In [110]:
import os
import shutil
import uuid

source_folder = "E:/Data/Cars/car_data/car_data/test"
destination_folder = "Train"  # Specify your destination folder

# Number of files to copy from each subfolder
num_files_to_copy = 30

for root, dirs, files in os.walk(source_folder):
    for subdir in dirs:
        # Get the path of the current subfolder
        subfolder_path = os.path.join(root, subdir)
        
        # Create a destination folder for the current subfolder
        dest_subfolder = os.path.join(destination_folder, subdir)
        os.makedirs(dest_subfolder, exist_ok=True)
        
        # List files in the subfolder
        subfolder_files = [f for f in os.listdir(subfolder_path) if os.path.isfile(os.path.join(subfolder_path, f))]
        
        # Copy the specified number of files from the subfolder
        for file_name in subfolder_files[:num_files_to_copy]:
            # Source file path
            source_file = os.path.join(subfolder_path, file_name)
            # Generate a UUID for the new file name
            new_filename = str(uuid.uuid4()) + os.path.splitext(file_name)[1]
            # Destination file path with the new filename
            dest_file = os.path.join(dest_subfolder, new_filename)
            # Copy the file and rename it to UUID
            shutil.copyfile(source_file, dest_file)
            print(f"Copied {file_name} from {subdir} and renamed to {new_filename}")

print("Copy operation completed.")


Copied 00128.jpg from Acura Integra Type R 2001 and renamed to 879a916e-68e5-41cd-ad41-3a8c1a7b7b0a.jpg
Copied 00130.jpg from Acura Integra Type R 2001 and renamed to 5cc8c1ba-76b9-4821-bfd8-d91073350f6e.jpg
Copied 00386.jpg from Acura Integra Type R 2001 and renamed to 72b9720a-cef6-4b24-abe5-c8781e13be56.jpg
Copied 00565.jpg from Acura Integra Type R 2001 and renamed to 4c06221f-1d26-4dbf-b713-ac03a768ef33.jpg
Copied 00711.jpg from Acura Integra Type R 2001 and renamed to 707fd65f-9cd3-4e2d-98d8-75d38fbeb43c.jpg
Copied 01002.jpg from Acura Integra Type R 2001 and renamed to 4c2da2a9-bdbc-4b30-a420-125dde8d8db3.jpg
Copied 01035.jpg from Acura Integra Type R 2001 and renamed to b329d1c2-42e4-4a86-bc71-7f93af0b8699.jpg
Copied 01176.jpg from Acura Integra Type R 2001 and renamed to 0747ee67-d267-48be-ab40-d4a81d5192aa.jpg
Copied 01326.jpg from Acura Integra Type R 2001 and renamed to b3c076ff-2929-42b4-b67d-0b1ef967d686.jpg
Copied 01864.jpg from Acura Integra Type R 2001 and renamed to b